In [2]:
from tagger.core.mongo.models.order import Order
import pandas as pd
from pytz import timezone

TZ = timezone("Asia/Seoul")
TARGETS = [
    # labels
    "617cc2b317fc2e5a1f9eaf94",
    "6178cbba17fc2e5a1f9ea7d2",
    "6178ce4117fc2e5a1f9ea7f5",
    "6181691526685ff051754dda",
    "6178d20c17fc2e5a1f9ea805",
    "617914d917fc2e5a1f9ea896",
    "6178d3ad17fc2e5a1f9ea80a",
    "6178c79a17fc2e5a1f9ea7c4",
    "617c1e5c17fc2e5a1f9eaf0a",
    "617ca0aa17fc2e5a1f9eaf66",
    "617fda0317fc2e5a1f9eb311",
#     pad
    "617fa15b17fc2e5a1f9eb26e",
    "617cbfd617fc2e5a1f9eaf8e",
    "6178da2a17fc2e5a1f9ea819",
    "6181691526685ff051754dda",
    
#     강영순 합배
    "617ca0aa17fc2e5a1f9eaf66",
    "617821d017fc2e5a1f9ea716"
]
CJ_COLUMNS = [
    "주문번호", "주문일", "수령자명", "수령자 우편번호", "수령자 연락처", "수령자주소", "상품명", "옵션명", "수량", "배송메모", "박스타입"
]



def get_cj_row(order):
    payment = order.get_payment()
    return [
        str(order.id),
        order.created.astimezone(TZ).date().isoformat().replace("-", "/"),
        payment.buyername,
        payment.buyerpostcode,
        f"{payment.buyermobile[:3]}-{payment.buyermobile[3:7]}-{payment.buyermobile[7:11]}",
        payment.buyeraddress,
        payment.name,
        ",".join([f"{item.size}/{item.quantity}EA" for item in order.orders]),
        1,
        order.memo,
        "소"
    ]

def get_cu_row(order):
    payment = order.get_payment()
    return [
        payment.buyername,
        payment.buyerpostcode,
        payment.buyeraddress,
        payment.buyeraddress,
        f"{payment.buyermobile[:3]}-{payment.buyermobile[3:7]}-{payment.buyermobile[7:11]}",
        "",
        order.memo,
        "선불"
    ]

def copy_cu():
    orders = Order.objects(id__in=TARGETS)
    df = pd.DataFrame([get_cu_row(order) for order in orders])
    df[1] = df[1].apply(lambda x: x.zfill(5))
    df.to_clipboard()

def make_cj():
    orders = Order.objects(id__in=TARGETS)
    df = pd.DataFrame([get_cj_row(order) for order in orders], columns=CJ_COLUMNS)
    df["수령자 우편번호"] = df["수령자 우편번호"].apply(lambda x: x.zfill(5))
    with open("../Downloads/cj_upload.xls", "wb") as file:
        df.to_excel(file, index=None)


In [3]:
make_cj()

FileNotFoundError: [Errno 2] No such file or directory: '~/cj_upload.xls'

'2021-10-17'